In [9]:
import pandas as pd

apfs_df = pd.read_csv(r'Initial_data/ac_power_from_storage.csv')
apts_df = pd.read_csv(r'Initial_data/ac_power_to_storage.csv')
gc_df = pd.read_csv(r'Initial_data/grid_consumption.csv')
gf_df = pd.read_csv(r'Initial_data/grid_feed.csv')
pp_df = pd.read_csv(r'Initial_data/pv_production.csv')


In [10]:
'''Convert the time column to a time type'''

apfs_df['time'] = pd.to_datetime(apfs_df['time'])
apts_df['time'] = pd.to_datetime(apts_df['time'])
gc_df['time'] = pd.to_datetime(gc_df['time'])
gf_df['time'] = pd.to_datetime(gf_df['time'])
pp_df['time'] = pd.to_datetime(pp_df['time'])
gc_df.to_csv(r'result/gen.csv', index=False)

'''Set the time as an index'''

apfs_df.set_index('time', inplace=True)
apts_df.set_index('time', inplace=True)
gc_df.set_index('time', inplace=True)
gf_df.set_index('time', inplace=True)
pp_df.set_index('time', inplace=True)
gc_df.to_csv(r'result/gen1.csv', index=False)


'''Convert the column of values to float type'''

apfs_df['ac_power_from_storage'] = apfs_df['ac_power_from_storage'].astype(float)
apts_df['ac_power_to_storage'] = apts_df['ac_power_to_storage'].astype(float)
gc_df['grid_consumption'] = gc_df['grid_consumption'].astype(float)
gf_df['grid_feed'] = gf_df['grid_feed'].astype(float)
pp_df['pv_production'] = pp_df['pv_production'].astype(float)
gc_df.to_csv(r'result/gen2.csv', index=False)


In [25]:
# Мердж данных. Здесь я использовал мердж для того, чтобы в нашем датасете произвести расчет нагрузки энергопотребления до момента ресэмплирования. По итогу в файле P_l получаем реальные данные(без подстановки нулей) с разрывами в значениях.

import pandas as pd
import numpy as np
from datetime import datetime


# Чтение всех датасетов
apfs_df = pd.read_csv(r'Initial_data/ac_power_from_storage.csv')
apts_df = pd.read_csv(r'Initial_data/ac_power_to_storage.csv')
gc_df = pd.read_csv(r'Initial_data/grid_consumption.csv')
gf_df = pd.read_csv(r'Initial_data/grid_feed.csv')
pp_df = pd.read_csv(r'Initial_data/pv_production.csv')

# Объединение датасетов
merged_df = apfs_df.merge(apts_df, on='time', how='outer') \
                  .merge(gc_df, on='time', how='outer') \
                  .merge(gf_df, on='time', how='outer') \
                  .merge(pp_df, on='time', how='outer')

# Преобразование столбца 'datetime' в datetime
merged_df['time'] = pd.to_datetime(merged_df['time'])

# Установка 'datetime' как индекс
merged_df.set_index('time', inplace=True)

# Рассчитываем P_l
merged_df['P_l'] = merged_df['pv_production'] + merged_df['grid_consumption'] + \
                   merged_df['ac_power_from_storage'] - merged_df['grid_feed'] - \
                   merged_df['ac_power_to_storage']

# Сохраняем результат в CSV
merged_df.to_csv(r'result/merged_df_with_P_l.csv')

# Выбираем только столбцы 'time' и 'P_l' из merged_df
result_df = merged_df[['P_l']]

# Сохраняем результат в CSV
result_df.to_csv(r'result/P_l.csv')





In [29]:
# Код аналогичен верхнему, только сделал ресэмплирование по 5 мин

from datetime import datetime
import pandas as pd
import numpy as np

# Чтение всех датасетов
apfs_df = pd.read_csv(r'Initial_data/ac_power_from_storage.csv')
apts_df = pd.read_csv(r'Initial_data/ac_power_to_storage.csv')
gc_df = pd.read_csv(r'Initial_data/grid_consumption.csv')
gf_df = pd.read_csv(r'Initial_data/grid_feed.csv')
pp_df = pd.read_csv(r'Initial_data/pv_production.csv')

# Объединение датасетов
merged_df = apfs_df.merge(apts_df, on='time', how='outer') \
                  .merge(gc_df, on='time', how='outer') \
                  .merge(gf_df, on='time', how='outer') \
                  .merge(pp_df, on='time', how='outer')

# Преобразование столбца 'datetime' в datetime
merged_df['time'] = pd.to_datetime(merged_df['time'])

# Установка 'datetime' как индекс
merged_df.set_index('time', inplace=True)

# Рассчитываем P_l
merged_df['P_l'] = merged_df['pv_production'] + merged_df['grid_consumption'] + \
                   merged_df['ac_power_from_storage'] - merged_df['grid_feed'] - \
                   merged_df['ac_power_to_storage']

# Сохраняем результат в CSV
merged_df.to_csv(r'result/merged_df_with_P_l.csv')

# Выбираем только столбцы 'time' и 'P_l' из merged_df
result_df = merged_df[['P_l']]

# Сохраняем результат в CSV
result_df.to_csv(r'result/P_l.csv')

# Create an empty DataFrame with the time index
index = pd.date_range(start_date, end_date, freq='5T')
df_general = pd.DataFrame(index=index)

# Сохраняем пустой DataFrame в CSV
df_general.to_csv(r'result/df_general.csv')


In [31]:
# Здесь мы убираем нули из нашего датасета
import pandas as pd

# Загрузите данные P_l из файла P_l.csv
df_P_l = pd.read_csv(r'result/P_l.csv')

# Преобразуйте столбец 'time' в datetime и установите его в качестве индекса
df_P_l['time'] = pd.to_datetime(df_P_l['time'])
df_P_l.set_index('time', inplace=True)

# Укажите частоту, с которой вы хотите пересэмплировать данные (например, '5T' для 5 минут)
resample_frequency = '5T'

# Пересэмплируйте данные с использованием суммирования для заполнения пропусков
df_resampled = df_P_l.resample(resample_frequency).sum()

# Замените нули на NaN
df_resampled['P_l'] = df_resampled['P_l'].where(df_resampled['P_l'] != 0, pd.NA)

# Сохраните результат в CSV
df_resampled.to_csv(r'result/filled_P_l.csv')




In [ ]:
# Твой старый код, где образуются нули в разрывах

from datetime import datetime

start_date = '2016-06-10 16:50:00+00:00'
end_date = '2023-09-09 00:00:00+00:00'

columns1 = ['time', 'P_l']
df_P_l = pd.DataFrame(columns=columns1)

columns2 = ['time', 'P_pv', 'P_fu', 'P_fs', 'P_tu', 'P_ts', 'P_l']
df_general = pd.DataFrame(columns=columns2)

iteration_date = start_date
time_delta = pd.Timedelta(minutes=5)

while iteration_date < end_date:
    P_pv = g_pp_df['pv_production'][iteration_date]
    P_fu = g_gc_df['grid_consumption'][iteration_date]
    P_fs = g_apfs_df['ac_power_from_storage'][iteration_date]
    P_tu = g_gf_df['grid_feed'][iteration_date]
    P_ts = g_apts_df['ac_power_to_storage'][iteration_date]
    
    P_l = P_pv + P_fu + P_fs - (P_tu + P_ts)
    
    new_row1 = {'time': iteration_date, 'P_l': P_l}
    new_row2 = {
        'time': iteration_date,
        'P_pv': P_pv, 'P_fu': P_fu, 
        'P_fs': P_fs, 'P_tu': P_tu, 
        'P_ts': P_ts, 'P_l': P_l
    }
    df_P_l = pd.concat([df_P_l, pd.DataFrame([new_row1])], ignore_index=True)
    df_general = pd.concat([df_general, pd.DataFrame([new_row2])], ignore_index=True)
    
    iteration_date = datetime.strptime(iteration_date, '%Y-%m-%d %H:%M:%S%z')
    iteration_date += time_delta
    iteration_date = iteration_date.strftime('%Y-%m-%d %H:%M:%S%z')

In [42]:
df_P_l['P_l'] = df_P_l['P_l'].round(3)
df_general['P_l'] = df_general['P_l'].round(3)

df_P_l.to_csv(r'result/P_l.csv', index=False)
df_general.to_csv(r'result/general.csv', index=False)

In [26]:
'''Updated version of code. Optimize the calculation loop using vectorized operations_код не актуален, он ускоряет процесс обработки твоего кода, но проставляет нули в разрывах''' 

import numpy as np

# Create an empty DataFrame with the time index
index = pd.date_range(start_date, end_date, freq='5T')
df_general = pd.DataFrame(index=index)
# Calculate the components
df_general['P_pv'] = g_pp_df['pv_production']
df_general['P_fu'] = g_gc_df['grid_consumption']
df_general['P_fs'] = g_apfs_df['ac_power_from_storage']
df_general['P_tu'] = g_gf_df['grid_feed']
df_general['P_ts'] = g_apts_df['ac_power_to_storage']

# Calculate P_l in one go
df_general['P_l'] = df_general['P_pv'] + df_general['P_fu'] + df_general['P_fs'] - (df_general['P_tu'] + df_general['P_ts'])

# Round the 'P_l' column
df_general['P_l'] = df_general['P_l'].round(3)
df_P_l['P_l'] = df_P_l['P_l'].round(3)

# Save the DataFrame to CSV
df_general.to_csv(r'result/general.csv')
df_P_l.to_csv(r'result/P_l.csv', index=False)



NameError: name 'g_pp_df' is not defined